In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models

from ds import *
from networks import *
from utils import *

## Get data loaders

In [ ]:
train_tfm = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])
train_dset = CxVAE_retino_Dset(
    csv_file='<path to>/retinopathy_small_balanced/balanced_train_labels.csv', 
    root_dir='<path to>/Datasets/retinopathy_small_balanced/balanced_subset_train3000_val100_test400/',
    tfm=train_tfm
)
val_dset = CxVAE_retino_Dset(
    csv_file='<path to>/Datasets/retinopathy_small_balanced/balanced_val_labels.csv', 
    root_dir='<path to>/Datasets/retinopathy_small_balanced/balanced_subset_train3000_val100_test400/'
)
test_dset = CxVAE_retino_Dset(
    csv_file='<path to>/Datasets/retinopathy_small_balanced/balanced_test_labels.csv', 
    root_dir='<path to>/Datasets/retinopathy_small_balanced/balanced_subset_train3000_val100_test400/'
)

train_loader = DataLoader(train_dset, batch_size=2, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dset, batch_size=2, shuffle=False, num_workers=8, pin_memory=False)
test_loader = DataLoader(test_dset, batch_size=2, shuffle=False, num_workers=8, pin_memory=False)

## Define model and pass to the training loop

In [ ]:
Net = AutoEncoder(3, 3, 8, 4, 32, 8)
print(Net)

In [ ]:
train_AE_loop(
    train_loader,
    val_loader,
    Net,
    n_epochs=100,
    init_lr=1e-4,
    eval_every = 5,
    dtype = torch.cuda.FloatTensor,
    device='cuda',
    ckpt_path = '../ckpt/AE_retino'
)

In [ ]:
Net.load_state_dict(torch.load('../ckpt/AE_retino_last.pth'))
eval_AE_loop(
    test_loader,
    Net,
    dtype = torch.cuda.FloatTensor,
    device='cuda',
)

In [ ]:
Net.load_state_dict(torch.load('../ckpt/AE_retino_last.pth'))
show_AE(
    test_loader,
    Net,
    dtype = torch.cuda.FloatTensor,
    device='cuda',
    n_show=3,
)